Install Streamlit

In [27]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2022-06-17 03:41:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  54.1MB/s    in 0.2s    

2022-06-17 03:41:04 (54.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



Unzip ngrok

In [29]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Insert token

In [30]:
get_ipython().system_raw('./ngrok authtoken 25l4MQV6C3Zfu3V5LpY8qvcdIfz_2bSbDFXqHDcCYsQdZtwj6')

Indicate port 8501

In [31]:
get_ipython().system_raw('./ngrok http 8501 &')

Tunel de conexion

In [35]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.7/json/__init__.py", line 296, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


App

In [33]:
%%writefile empleados.py
import pandas as pd
import numpy as np
import streamlit as st
import codecs
import re

DATA_URL = 'https://raw.githubusercontent.com/Psychodivto/Bicicletas/main/citibike-tripdata.csv'
COLUMNA = 'started_at'

st.title('Cicle Rides in NYC')
sidebar = st.sidebar

st.header('Diego Ivan Perez Conde')
st.subheader('ISW 602')
st.text('Aplicación web encargada de analizar los dados recopilados del archivo CVS citibike-tripdata.csv')

@st.cache
def cargar_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows, encoding_errors='ignore')
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[COLUMNA] = pd.to_datetime(data[COLUMNA])
    return data

checkbox_data = sidebar.checkbox("Mostrar todos los datos")
if checkbox_data:
    estado = st.text('Cargando...')
    data = cargar_data(500)    
    estado.text("¡Cargado! (usando st.cache)")
    st.subheader('Todos los datos')
    st.dataframe(data)

checkbox_hora = sidebar.checkbox("Mostrar los datos por hora")
if checkbox_hora:
    estado = st.text('Cargando...')
    data = cargar_data(500)
    estado.text("¡Cargado! (usando st.cache)")
    st.subheader('Numero de recorridos por hora')
    values = np.histogram(data[COLUMNA].dt.hour, bins=24, range=(0,24))[0]
    st.bar_chart(values)


filtro_hora = sidebar.slider('HORA', 0, 23, 17)
if filtro_hora:
    estado = st.text('Cargando...')
    data = cargar_data(500)
    estado.text("¡Cargado! (usando st.cache)")
    data_rename = data.rename(columns = {'start_lat': 'lat', 'start_lng': 'lon'}, inplace = False)
    filtrar_data = data_rename[data_rename[COLUMNA].dt.hour == filtro_hora]
    st.subheader('Mapa de los recorridos iniciados a las %s:00' % filtro_hora)
    st.map(filtrar_data)


Overwriting empleados.py


Conexion

In [9]:
!streamlit run /content/empleados.py

2022-06-17 03:26:19.006 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.75.255.182:8501

  Stopping...
  Stopping...


Install Sidebar